# Aggregates ➕➕

We have used some Spark SQL before, today we will practice with it some more, specifically we will focus on **aggregation operations**.

## What will you learn in this course? 🧐🧐

This course will walk you through the mechanics of aggregation with Spark SQL. Here's the outline:

* Import functions
* Data
    * Schema
    * Date
* GroupedData
    * `.mean()`
    * `.sum()`
    * `.count()`
    * Other aggregations
    * `.agg(exprs)`
        * exprs as dict
        * exprs as list
        * advanced expressions

In this notebook we will be using data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Online%20Retail).  

We'll actually use 👉 [this version](https://www.kaggle.com/carrie1/ecommerce-data) 👈 of the dataset for convenience reasons (csv format).

In [ ]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/data.csv"

sales = spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath)

## Import functions 🔧🔧

Before we get started, let's import PySpark's SQL functions. This module contains all the functions you'll need to do everything you could do with regular SQL using PySpark!

In [ ]:
from pyspark.sql import functions as F

## Our data 📊📊
Let's take a quick look at the first few rows of our dataset to get a feel of what's in there before we get started.

In [ ]:
sales.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

We have 8 columns here: 

* InvoiceNo: the identification number for an order
* StockCode: the identification of the product
* Description: a description of the product
* Quantity: The amount of product purchased
* InvoiceDate: The day and time the order was placed
* UnitPrice: The price for one unit of the product
* CustomerID: The identification number of the customer
* Country: The customer's country of residence

We're **ready to roll!**

### Schema 📋
Let's take a look of the data schema of this DataFrame, make sure everyting is in order.

In [ ]:
sales.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: integer (nullable = true)
-- Country: string (nullable = true)

In [ ]:
sales.describe().toPandas()

Out[41]:

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,541909,541909,540455,541909,541909,541909,406829,541909
1,mean,559965.752026781,27623.240210938104,20713.0,9.55224954743324,None,4.6111136260897085,15287.690570239585,None
2,stddev,13428.417280796779,16799.737628427658,None,218.08115785023438,None,96.75985306117963,1713.6003033215982,None
3,min,536365,10002,4 PURPLE FLOCK DINNER CANDLES,-80995,1/10/2011 10:04,-11062.06,12346,Australia
4,max,C581569,m,wrongly sold sets,80995,9/9/2011 9:52,38970.0,18287,Unspecified


Looks like some columns have incoherent values, let's clean them up a little before we move on with the analysis:

In [ ]:
sales = sales.filter((F.col("Quantity")>0) & (F.col("UnitPrice")>0)) 
sales.describe().toPandas()
# filter out quantities and unitprices below 0

Out[48]:

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,530104,530104,530104,530104,530104,530104,397884,530104
1,mean,559981.4746888812,27591.351654656588,None,10.542037034242338,None,3.907625247122361,15294.423452564064,None
2,stddev,13430.049737665431,16756.848658106595,None,155.52412351063666,None,35.915681104255405,1713.1415604398567,None
3,min,536365,10002,4 PURPLE FLOCK DINNER CANDLES,1,1/10/2011 10:32,0.001,12346,Australia
4,max,A563185,m,ZINC WIRE SWEETHEART LETTER TRAY,80995,9/9/2011 9:52,13541.33,18287,Unspecified


Seems like the data Schema did not get any of the types wrong, except for the date, let's convert the date to the proper format before we are ready to move forward.

### Date 📆

Let's work on the date data a little so it's in the proper format.

Two functions are very useful for handling dates `F.to_timestamp(col, format="")` which converts character strings to [timestamp type](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.TimestampType.html#pyspark.sql.types.TimestampType) and `F.to_date(col,format="")`which converts character strings to [date type](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.DateType.html#pyspark.sql.types.DateType).

Here are links to the documentation if you want to go further: [to_date](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.to_date.html), [to_timestamp](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.to_timestamp.html).

The function to_timestamp is able to convert the string format date to a timestamp format! All you have to do is write the proper format argument:

In [ ]:
sales = sales.withColumn("InvoiceDateClean", F.to_timestamp(F.col("InvoiceDate"), format="M/d/y H:m"))
sales.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country| InvoiceDateClean|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom|2010-12-01 08:26:00|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|2010-12-01 08:26:00|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom|2010-12-01 08:26:00|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|2010-12-01 08:26:00|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|2010-12-01 08:26:00|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-------------------+
only showing top 5 rows

Let's drop the previous date column:

In [ ]:
sales = sales.drop(F.col("InvoiceDate"))

Once the column is in date format, it is possible to extract specific information from the date:

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, dayofyear, weekofyear

sales.select(year(F.col(InvoiceDateClean))).show(5)

## GroupedData 📊📊

What's the type of `sales.groupBy('CustomerID')`?

In [ ]:
type(sales.groupBy('CustomerID'))

Out[51]: pyspark.sql.group.GroupedData

A `GroupedData`, here's the link to the [documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData).

We'll go through what we can do with this.

### .mean()

`.mean()` is an alias to `.avg()`

Compute the average of all numeric columns with `.avg()`

In [ ]:
sales.groupBy('CustomerID').avg().show(5)

+----------+------------------+------------------+---------------+
CustomerID| avg(Quantity)| avg(UnitPrice)|avg(CustomerID)|
+----------+------------------+------------------+---------------+
 17420| 8.833333333333334| 3.881333333333332| 17420.0|
 16503| 6.773809523809524| 4.173571428571428| 16503.0|
 15727|10.149006622516556|3.2481456953642383| 15727.0|
 17389|35.737089201877936|5.6888262910798115| 17389.0|
 15447| 9.444444444444445|3.1211111111111114| 15447.0|
+----------+------------------+------------------+---------------+
only showing top 5 rows

In [ ]:
sales.groupBy('CustomerID').mean().show(5)

+----------+------------------+------------------+---------------+
CustomerID| avg(Quantity)| avg(UnitPrice)|avg(CustomerID)|
+----------+------------------+------------------+---------------+
 17420| 8.833333333333334| 3.881333333333332| 17420.0|
 16503| 6.773809523809524| 4.173571428571428| 16503.0|
 15727|10.149006622516556|3.2481456953642383| 15727.0|
 17389|35.737089201877936|5.6888262910798115| 17389.0|
 15447| 9.444444444444445|3.1211111111111114| 15447.0|
+----------+------------------+------------------+---------------+
only showing top 5 rows

### `.sum()`

In [ ]:
sales.groupBy('customerID').sum().show(5)

+----------+-------------+------------------+---------------+
customerID|sum(Quantity)| sum(UnitPrice)|sum(CustomerID)|
+----------+-------------+------------------+---------------+
 17420| 265|116.43999999999997| 522600|
 16503| 569| 350.58| 1386252|
 15727| 3065| 980.9399999999999| 4749554|
 17389| 7612|1211.7199999999998| 3703857|
 15447| 85|28.090000000000003| 139023|
+----------+-------------+------------------+---------------+
only showing top 5 rows

We can select a specific column...

In [ ]:
sales.groupBy('customerID').mean('Quantity').show(5)

+----------+------------------+
customerID| avg(Quantity)|
+----------+------------------+
 17420| 8.833333333333334|
 16503| 6.773809523809524|
 15727|10.149006622516556|
 17389|35.737089201877936|
 15447| 9.444444444444445|
+----------+------------------+
only showing top 5 rows

And several columns at once...

In [ ]:
sales.groupBy('customerID').mean('Quantity', 'UnitPrice').show(5)

+----------+------------------+------------------+
customerID| avg(Quantity)| avg(UnitPrice)|
+----------+------------------+------------------+
 17420| 8.833333333333334| 3.881333333333332|
 16503| 6.773809523809524| 4.173571428571428|
 15727|10.149006622516556|3.2481456953642383|
 17389|35.737089201877936|5.6888262910798115|
 15447| 9.444444444444445|3.1211111111111114|
+----------+------------------+------------------+
only showing top 5 rows

This won't work with a list, a tuple or a generator. These need to be **unpacked**:

In [ ]:
# This will fail without unpacking!
col_list = ['Quantity', 'UnitPrice']
sales.groupBy('customerID').mean(*col_list).show(5)

+----------+------------------+------------------+
customerID| avg(Quantity)| avg(UnitPrice)|
+----------+------------------+------------------+
 17420| 8.833333333333334| 3.881333333333332|
 16503| 6.773809523809524| 4.173571428571428|
 15727|10.149006622516556|3.2481456953642383|
 17389|35.737089201877936|5.6888262910798115|
 15447| 9.444444444444445|3.1211111111111114|
+----------+------------------+------------------+
only showing top 5 rows

### `.count()`

`count()` is a bit different, it doesn't apply to any column, it count the number of rows in the DataFrame.  
This is different from pandas, where `.count()` count the number of non-null values.

In [ ]:
sales.groupBy('customerID').count().show(5)

+----------+-----+
customerID|count|
+----------+-----+
 17420| 30|
 16503| 84|
 15727| 302|
 17389| 213|
 15447| 9|
+----------+-----+
only showing top 5 rows

### Other aggregations 🛠️

Other aggregation functions include: `.max()`, `.min()` and `.pivot()`.

In [ ]:
sales.groupBy('customerID').pivot('Quantity').count().limit(5).toPandas()
# this creates a pivot table where rows represent cutomer ids, columns
# represent quantities and the values correspond to the result of the aggregation
# function

Out[59]:

,customerID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1296,1300,1350,1356,1394,1400,1404,1412,1428,1440,1488,1500,1515,1540,1600,1728,1788,1820,1824,1878,1900,1930,1944,1992,2000,2040,2100,2160,2400,2592,2700,2880,3000,3114,3186,3906,4300,4800,74215,80995
0,16503,5.0,15,6.0,7.0,NaN,17,NaN,11.0,NaN,5.0,NaN,14,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15727,22.0,27,11.0,40.0,5.0,57,NaN,11.0,2.0,39.0,2.0,35,NaN,NaN,NaN,1.0,NaN,1.0,NaN,4.0,1.0,NaN,NaN,22.0,7.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15447,NaN,1,NaN,NaN,1.0,1,NaN,1.0,NaN,NaN,NaN,4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14450,4.0,5,3.0,10.0,2.0,6,1.0,1.0,NaN,2.0,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17420,1.0,4,NaN,NaN,NaN,4,NaN,NaN,NaN,16.0,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### `.agg(exprs)`

And a last one: `.agg(exprs)`. This one is a bit meta, it will compute the aggregate of a function given as parameter.  

From the [Documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData.agg):

> The available aggregate functions are `avg`, `max`, `min`, `sum`, `count`.

It makes it possible to easily compute more complicated aggregations.

#### Single dict mapping str to str 

If `exprs` is a single dict mapping from string to string, then the key is the column to perform aggregation on, and the value is the aggregate function.

In [ ]:
agg_dict = {'Quantity': 'mean', 'UnitPrice': 'sum'}
sales.groupBy('customerId').agg(agg_dict).show(5)

+----------+------------------+------------------+
customerId| sum(UnitPrice)| avg(Quantity)|
+----------+------------------+------------------+
 17420|116.43999999999997| 8.833333333333334|
 16861|18.730000000000004| 16.625|
 16503| 361.08| 6.593023255813954|
 15727| 980.9399999999999|10.149006622516556|
 17389| 1273.57|33.169642857142854|
+----------+------------------+------------------+
only showing top 5 rows

Can use `'*'` if calling `.count()`. But not with others.

In [ ]:
sales.groupBy('customerId').agg({'*': 'count'}).show(5)

+----------+--------+
customerId|count(1)|
+----------+--------+
 17420| 30|
 16861| 8|
 16503| 86|
 15727| 302|
 17389| 224|
+----------+--------+
only showing top 5 rows

#### List of aggregate expressions

Alternatively, `exprs` can also be a "list" of aggregate Column expressions.  

This requires **unpacking**:

In [ ]:
agg_exprs = (F.mean('Quantity'), F.sum('UnitPrice'))
sales.groupBy('customerId').agg(*agg_exprs).show()

+----------+------------------+------------------+
customerId| avg(Quantity)| sum(UnitPrice)|
+----------+------------------+------------------+
 17420| 8.833333333333334|116.43999999999997|
 16861| 16.625|18.730000000000004|
 16503| 6.593023255813954| 361.08|
 15727|10.149006622516556| 980.9399999999999|
 17389|33.169642857142854| 1273.57|
 15447| 9.444444444444445|28.090000000000003|
 14450| 6.025|111.62000000000002|
 15100| 9.666666666666666| 65.7|
 12471| 15.0| 2266.869999999999|
 16916| 3.076923076923077| 317.52|
 14514| 8.068493150684931| 235.5300000000001|
 16500|11.181818181818182| 188.32|
 12626|10.912052117263844| 871.9999999999995|
 18161| 10.0| 294.16|
 15967| 2.511904761904762|240.88000000000002|
 17809| 31.484375| 897.1900000000002|
 15738|16.857954545454547|372.82000000000005|
 17044| 48.7|38.419999999999995|
 12393| 12.75| 145.9|
 16791| 2.357142857142857| 943.13|
+----------+------------------+------------------+
only showing top 20 rows

Now, you can alias, and that's because `.alias()` also returns a _Column Expression_.

In [ ]:
type(F.mean('Quantity').alias('meanQuantity'))

Out[15]: pyspark.sql.column.Column

In [ ]:
sales.groupBy('customerId').agg(
    F.mean('Quantity').alias('meanQuantity'),
    F.sum('UnitPrice').alias('totalPrice')
).show(5)

+----------+------------------+------------------+
customerId| meanQuantity| totalPrice|
+----------+------------------+------------------+
 17420| 8.833333333333334|116.43999999999997|
 16861| 16.625|18.730000000000004|
 16503| 6.593023255813954| 361.08|
 15727|10.149006622516556| 980.9399999999999|
 17389|33.169642857142854| 1273.57|
+----------+------------------+------------------+
only showing top 5 rows

#### Advanced expressions ⚗️

It possible to calculate more complicated expressions thanks to `.agg()`:

In [ ]:
sales.groupBy('customerId').agg((F.sum(F.col("UnitPrice"))/F.sum(F.col("Quantity"))).alias("avg_unit_price")).show(5) 

+----------+-------------------+
customerId| avg_unit_price|
+----------+-------------------+
 17420|0.43939622641509424|
 16503| 0.6161335676625659|
 15727|0.32004567699836867|
 17389| 0.1591854965843405|
 15447| 0.3304705882352942|
+----------+-------------------+
only showing top 5 rows